## install & import packages

#### Press Kernel above -> restart & run all

In [ ]:
pip install pandas_ta

In [ ]:
pip install cbpro

In [ ]:
pip install dataclasses

In [ ]:
pip install matplotlib

In [ ]:
import json
import pandas_ta as ta
import cbpro
import pandas as pd
import matplotlib.pyplot as plt

## public coinbase query

In [ ]:
public_client = cbpro.PublicClient() #opens a client to query PUBLIC coinbase api

In [ ]:
df = pd.read_json(json.dumps(public_client.get_product_historic_rates('BTC-USD', granularity=900))) #Granularity choices: 60 (minute), 300 (5 minutes), 900 (15 mintes), 3600 (1 hour), 21600 (6 hours), 86400 (1 day)

In [ ]:
df.columns = ['time', 'low', 'high', 'open', 'close', 'volume'] #label columns
df_time = df["time"]
df_low = df["low"]
df_high = df["high"]
df_open = df["open"]
df_close = df["close"]
df_volume = df["volume"]

In [ ]:
rsi = ta.rsi(df_close, 14)
sma50 = ta.sma(df_close, 50)

#### run below to plot the above historic data

In [ ]:
ax1 = plt.subplot(411)
plt.plot(df_close, label="close")
plt.plot(sma50, label="sma50")
plt.legend()
plt.xlabel('time')
plt.ylabel('close')

plt.subplot(412, sharex=ax1)
plt.plot(rsi, label="rsi")
plt.legend()
plt.ylabel('rsi')

plt.show()

## Websocket stream

In [ ]:
api_key = 'b530f8d3d0c88c44c63300fdfcdede40'
api_secret = 'WS9U7h4ZCLS/3nAiMqtbWWzlYWmcE57AlvZ9bsoW60eCuIiGTcEt1FVQfnD91vWPVUj6icvRjRGAlUtqe187HQ=='
api_passphrase = 'pickysandbox'

In [ ]:
class TextWebsocketClient(cbpro.WebsocketClient):
    def on_open(self):
        self.url = 'wss://ws-feed-public.sandbox.pro.coinbase.com'
        self.message_count = 0
        
    def on_message(self,msg):
        self.message_count += 1
        msg_type = msg.get('type',None)
        if msg_type == 'ticker':
            time_val = msg.get('time',('-'*27))
            price_val = msg.get('price',None)
            price_val = float(price_val) if price_val is not None else 'None'
            product_id = msg.get('product_id',None)

            print(f"{time_val:30} {price_val:.3f} {product_id}\tchannel type:{msg_type}")
        
    def on_close(self):
        print(f"<---Websocket connection closed--->\n\tTotal messages: {self.message_count}")

#### run below to start stream

In [ ]:
stream = TextWebsocketClient(products=['BTC-USD'],channels=['ticker'])
stream.start()

#### run below to close stream

In [ ]:
stream.close()